In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import col,struct,when, lit
from pyspark.sql import Row

print("imported")

# spark session
spark = SparkSession.builder.master("local[1]").appName('Rowschema').getOrCreate()

imported


In [6]:
jsondata = spark.read.format("json").option("multiline", "true").load("file:///D:/Data Analytics applications/Eclipse Projects/data_files/jl.json")

jsondata.show()
jsondata.printSchema()

+---+---------+--------------------+-------+
| id|institute|            location|trainer|
+---+---------+--------------------+-------+
|  1| zeyobron|{Hyderabad, chennai}|    sai|
+---+---------+--------------------+-------+

root
 |-- id: long (nullable = true)
 |-- institute: string (nullable = true)
 |-- location: struct (nullable = true)
 |    |-- permanentLocation: string (nullable = true)
 |    |-- temporaryLocation: string (nullable = true)
 |-- trainer: string (nullable = true)



In [7]:
destruct = jsondata.select("id", "institute", "location.permanentLocation", "location.temporaryLocation" , "trainer")

destruct.show()
destruct.printSchema()

+---+---------+-----------------+-----------------+-------+
| id|institute|permanentLocation|temporaryLocation|trainer|
+---+---------+-----------------+-----------------+-------+
|  1| zeyobron|        Hyderabad|          chennai|    sai|
+---+---------+-----------------+-----------------+-------+

root
 |-- id: long (nullable = true)
 |-- institute: string (nullable = true)
 |-- permanentLocation: string (nullable = true)
 |-- temporaryLocation: string (nullable = true)
 |-- trainer: string (nullable = true)



In [8]:
jl1 = spark.read.format("json").option("multiline", "true").load("file:///D:/Data Analytics applications/Eclipse Projects/data_files/jl1.json")

jl1.show()
jl1.printSchema()

+---+---------+-------+--------------------+
| id|institute|trainer|               users|
+---+---------+-------+--------------------+
|  1| zeyobron|    sai|{{vasu, liyan, tej}}|
+---+---------+-------+--------------------+

root
 |-- id: long (nullable = true)
 |-- institute: string (nullable = true)
 |-- trainer: string (nullable = true)
 |-- users: struct (nullable = true)
 |    |-- user: struct (nullable = true)
 |    |    |-- first: string (nullable = true)
 |    |    |-- last: string (nullable = true)
 |    |    |-- middle: string (nullable = true)



In [9]:
jldestruct = jl1.select("id", "institute", "users.user.*", "trainer")

jldestruct.show()
jldestruct.printSchema()

+---+---------+-----+-----+------+-------+
| id|institute|first| last|middle|trainer|
+---+---------+-----+-----+------+-------+
|  1| zeyobron| vasu|liyan|   tej|    sai|
+---+---------+-----+-----+------+-------+

root
 |-- id: long (nullable = true)
 |-- institute: string (nullable = true)
 |-- first: string (nullable = true)
 |-- last: string (nullable = true)
 |-- middle: string (nullable = true)
 |-- trainer: string (nullable = true)



In [10]:
jln = spark.read.format("json").option("multiline", "true").load("file:///D:/Data Analytics applications/Eclipse Projects/data_files/jln.json")

jln.show()
jln.printSchema()

+--------------------+---+---------+
|            Students| id|institute|
+--------------------+---+---------+
|[Rajesh, Monika, ...|  1| zeyobron|
+--------------------+---+---------+

root
 |-- Students: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: long (nullable = true)
 |-- institute: string (nullable = true)



In [11]:
from pyspark.sql.functions import *
explodedata = jln.withColumn("Students", explode("Students"))
explodedata.show()
explodedata.printSchema()

+--------+---+---------+
|Students| id|institute|
+--------+---+---------+
|  Rajesh|  1| zeyobron|
|  Monika|  1| zeyobron|
|    Ravi|  1| zeyobron|
+--------+---+---------+

root
 |-- Students: string (nullable = true)
 |-- id: long (nullable = true)
 |-- institute: string (nullable = true)



In [12]:
jkn = spark.read.format("json").option("multiline", "true").load("file:///D:/Data Analytics applications/Eclipse Projects/data_files/jkn.json")

jkn.show()
jkn.printSchema()

+--------------------+---+---------+
|            Students| id|institute|
+--------------------+---+---------+
|[{{46, rajesh}}, ...|  1| zeyobron|
+--------------------+---+---------+

root
 |-- Students: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- user: struct (nullable = true)
 |    |    |    |-- age: string (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |-- id: long (nullable = true)
 |-- institute: string (nullable = true)



In [15]:
jknexplodedata = jkn.withColumn("Students", explode("Students")).select("id", "institute", "Students.user.*" )
jknexplodedata.show()
jknexplodedata.printSchema()

+---+---------+---+------+
| id|institute|age|  name|
+---+---------+---+------+
|  1| zeyobron| 46|rajesh|
|  1| zeyobron| 24|  mani|
+---+---------+---+------+

root
 |-- id: long (nullable = true)
 |-- institute: string (nullable = true)
 |-- age: string (nullable = true)
 |-- name: string (nullable = true)



In [16]:
structgen = jknexplodedata.select("id", struct("institute", "age", "name"))
structgen.show()
structgen.printSchema()

+---+----------------------------+
| id|struct(institute, age, name)|
+---+----------------------------+
|  1|        {zeyobron, 46, ra...|
|  1|        {zeyobron, 24, mani}|
+---+----------------------------+

root
 |-- id: long (nullable = true)
 |-- struct(institute, age, name): struct (nullable = false)
 |    |-- institute: string (nullable = true)
 |    |-- age: string (nullable = true)
 |    |-- name: string (nullable = true)



In [36]:
import urllib.request
import ssl
context = ssl.create_default_context()
context.check_hostname = False
context.verify_mode = ssl.CERT_NONE
url = "https://randomuser.me/api/0.8/?results=1"
response = urllib.request.urlopen(url, context=context).read().decode('utf-8')
urlstring = response
print(urlstring)

{
    "results": [
        {
            "user": {
                "gender": "male",
                "name": {
                    "title": "monsieur",
                    "first": "tristan",
                    "last": "brun"
                },
                "location": {
                    "street": "1583 rue pasteur",
                    "city": "echichens",
                    "state": "obwalden",
                    "zip": 4374
                },
                "email": "tristan.brun@example.com",
                "username": "orangemeercat531",
                "password": "america",
                "salt": "xxTV3ueD",
                "md5": "d11ff867ca87aef568aeea9774ea011f",
                "sha1": "6febf173d5e5656ebc8f5f26db37a77a9b51a016",
                "sha256": "ba5027fa20c29f8d5cda0a327c2433a81208ceb21fc903aee10e6a9197dfe549",
                "registered": 935043702,
                "dob": 1029121072,
                "phone": "(846)-579-4121",
                "cell": "

In [35]:
# spark context
conf = SparkConf().setAppName("Spark").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf = conf)
sc.setLogLevel("ERROR")

rdd = sc.parallelize([urlstring])
df = spark.read.json("https://randomuser.me/api/0.8/?results=1")
df.show()

Py4JJavaError: An error occurred while calling o225.json.
: java.lang.UnsupportedOperationException
	at org.apache.hadoop.fs.http.AbstractHttpFileSystem.listStatus(AbstractHttpFileSystem.java:95)
	at org.apache.hadoop.fs.http.HttpsFileSystem.listStatus(HttpsFileSystem.java:23)
	at org.apache.hadoop.fs.Globber.listStatus(Globber.java:128)
	at org.apache.hadoop.fs.Globber.doGlob(Globber.java:291)
	at org.apache.hadoop.fs.Globber.glob(Globber.java:202)
	at org.apache.hadoop.fs.FileSystem.globStatus(FileSystem.java:2124)
	at org.apache.spark.deploy.SparkHadoopUtil.globPath(SparkHadoopUtil.scala:254)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$3(DataSource.scala:737)
	at org.apache.spark.util.ThreadUtils$.$anonfun$parmap$2(ThreadUtils.scala:393)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Futu

In [38]:
from urllib.request import Request, urlopen
onlineData = 'https://randomuser.me/api/0.8/?results=10'

# read the online data file
httpData = urlopen(onlineData).read().decode('utf-8')

# convert into RDD
rdd = sc.parallelize([httpData])

# create a Dataframe
jsonDF = spark.read.json(httpData)
jsonDF.show()


IllegalArgumentException: java.net.URISyntaxException: Relative path in absolute URI: {
    "results":%20%5B%0A%20%20%20%20%20%20%20%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%22user%22:%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%22gender%22:%20%22male%22,%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%22name%22:%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%22title%22:%20%22mr%22,%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%22first%22:%20%22antonio%22,%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%22last%22:%20%22cortes%22%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%7D,%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%22location%22:%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%22street%22:%20%226744%20avenida%20de%20castilla%22,%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%22city%22:%20%22pamplona%22,%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20

# Project

In [39]:
jsondata = spark.read.format("json").option("multiline", "true").load("file:///D:/Data Analytics applications/Eclipse Projects/data_files/project.json")
jsondata.show()
jsondata.printSchema()


+-----------+--------------------+------------------+-------+
|nationality|             results|              seed|version|
+-----------+--------------------+------------------+-------+
|         CH|[{{756.VULG.RZAJ....|6ab0db8063d4960303|    0.8|
+-----------+--------------------+------------------+-------+

root
 |-- nationality: string (nullable = true)
 |-- results: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- user: struct (nullable = true)
 |    |    |    |-- AVS: string (nullable = true)
 |    |    |    |-- cell: string (nullable = true)
 |    |    |    |-- dob: long (nullable = true)
 |    |    |    |-- email: string (nullable = true)
 |    |    |    |-- gender: string (nullable = true)
 |    |    |    |-- location: struct (nullable = true)
 |    |    |    |    |-- city: string (nullable = true)
 |    |    |    |    |-- state: string (nullable = true)
 |    |    |    |    |-- street: string (nullable = true)
 |    |    |    |    |-- zip:

In [46]:
data2 = jsondata.withColumn("results", explode("results"))
data2.show()
data2.printSchema()

+-----------+--------------------+------------------+-------+
|nationality|             results|              seed|version|
+-----------+--------------------+------------------+-------+
|         CH|{{756.VULG.RZAJ.8...|6ab0db8063d4960303|    0.8|
|         CH|{{756.YBBE.WNNN.0...|6ab0db8063d4960303|    0.8|
|         CH|{{756.CLOO.XNYI.3...|6ab0db8063d4960303|    0.8|
|         CH|{{756.VZBP.TMGT.3...|6ab0db8063d4960303|    0.8|
|         CH|{{756.FIKW.ADKM.4...|6ab0db8063d4960303|    0.8|
|         CH|{{756.MKQV.JCEM.6...|6ab0db8063d4960303|    0.8|
|         CH|{{756.QZEP.ZCDV.1...|6ab0db8063d4960303|    0.8|
|         CH|{{756.BDUZ.PTAM.8...|6ab0db8063d4960303|    0.8|
|         CH|{{756.HONT.DEDG.3...|6ab0db8063d4960303|    0.8|
|         CH|{{756.VUSX.MHNW.8...|6ab0db8063d4960303|    0.8|
+-----------+--------------------+------------------+-------+

root
 |-- nationality: string (nullable = true)
 |-- results: struct (nullable = true)
 |    |-- user: struct (nullable = true)
 | 

In [45]:
data3 = data2.select("results.user.username", "results.user.AVS", "results.user.cell", "results.user.dob", "results.user.email", "results.user.gender", "results.user.location.*", "results.user.name.*", "results.user.password", "results.user.phone", "results.user.picture.*", "results.user.registered", "seed", "version", "nationality")
data3.show(1)
data3.printSchema()

+----------+----------------+--------------+---------+--------------------+------+-------------+------+------------------+----+--------+------+------------+--------+--------------+--------------------+--------------------+--------------------+----------+------------------+-------+-----------+
|  username|             AVS|          cell|      dob|               email|gender|         city| state|            street| zip|   first|  last|       title|password|         phone|               large|              medium|           thumbnail|registered|              seed|version|nationality|
+----------+----------------+--------------+---------+--------------------+------+-------------+------+------------------+----+--------+------+------------+--------+--------------+--------------------+--------------------+--------------------+----------+------------------+-------+-----------+
|redfrog197|756.VULG.RZAJ.83|(992)-543-7675|723370357|victoire.dufour@e...|female|montpreveyres|zürich|1559 rue de bon